In [ ]:
import time
import sys
import os
import config
import config_runtime
import subprocess
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

In [ ]:
# — Asegurar UTF-8 en stdout —
if hasattr(sys.stdout, "reconfigure"):
    sys.stdout.reconfigure(encoding="utf-8", errors="replace")

def safe_print(text):
    """Imprime sin romper con caracteres Unicode."""
    try:
        print(text)
    except UnicodeEncodeError:
        enc = sys.stdout.encoding or "utf-8"
        fixed = text.encode(enc, errors="replace").decode(enc)
        print(fixed)

def ejecutar_script(path):
    """Lanza un script Python y captura stdout/stderr."""
    safe_print(f"\n🚀 Ejecutando: {path}")
    env = os.environ.copy()
    env["PYTHONIOENCODING"] = "utf-8"
    result = subprocess.run(
        ["python", path],
        env=env,
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        safe_print("✅ Script OK")
    else:
        safe_print(f"⚠️ Exit code {result.returncode}")
    safe_print("🔹 STDOUT:")
    safe_print(result.stdout)
    safe_print("🔸 STDERR:")
    safe_print(result.stderr)

def save_config_runtime():
    lines = [
        f"usuario = {config_runtime.usuario!r}\n",
        f"reporte = {config_runtime.reporte!r}\n",
        f"pbit_queue = {config_runtime.pbit_queue!r}\n",
    ]
    with open("config_runtime.py", "w", encoding="utf-8") as f:
        f.writelines(lines)

from watchdog.events import FileSystemEventHandler
import os, config_runtime

class PbitHandler(FileSystemEventHandler):
    def on_created(self, event):
        if event.is_directory or not event.src_path.lower().endswith(".pbit"):
            return

        # only keep the filename, not the full path:
        nombre = os.path.basename(event.src_path)

        # avoid duplicates
        if nombre in config_runtime.pbit_queue:
            return

        config_runtime.pbit_queue.append(nombre)
        save_config_runtime()
        safe_print(f"📑 Cola actualizada: {config_runtime.pbit_queue}")


In [ ]:
import os
import sys
import subprocess

# — Asegurar UTF-8 en stdout —
if hasattr(sys.stdout, "reconfigure"):
    sys.stdout.reconfigure(encoding="utf-8", errors="replace")

def safe_print(text: str):
    """Imprime de forma segura, reemplazando caracteres no codificables."""
    try:
        print(text)
    except UnicodeEncodeError:
        enc = sys.stdout.encoding or "utf-8"
        print(text.encode(enc, errors="replace").decode(enc))

def ejecutar_script(path: str) -> str:
    """
    Executes the script at 'path', captures stdout/stderr as raw bytes,
    and then manually decodes them using UTF-8 with error replacement.
    This provides a more robust way to handle encoding issues.
    Returns a formatted string with the complete output.
    """
    safe_print(f"\n🚀 Ejecutando: {path}")
    
    # Set the environment variable to encourage the child process to use UTF-8.
    env = os.environ.copy()
    env["PYTHONIOENCODING"] = "utf-8"

    # Run the subprocess without text=True to receive raw bytes.
    proc = subprocess.run(
        ["python", path],
        env=env,
        capture_output=True # Captures stdout and stderr
    )

    # Manually decode the byte output, replacing any problematic characters.
    out = proc.stdout.decode("utf-8", errors="replace")
    err = proc.stderr.decode("utf-8", errors="replace")

    # Build the final output string for printing.
    output_lines = []
    if proc.returncode != 0:
        output_lines.append(f"⚠️ Exit code {proc.returncode}")
    else:
        output_lines.append("✅ Script OK")

    output_lines.append("🔹 STDOUT:")
    output_lines.append(out if out.strip() else "(sin salida)")
    
    output_lines.append("🔸 STDERR:")
    output_lines.append(err if err.strip() else "(sin errores)")

    return "\n".join(output_lines)

def save_config_runtime():
    lines = [
        f"usuario = {config_runtime.usuario!r}\n",
        f"reporte = {config_runtime.reporte!r}\n",
        f"pbit_queue = {config_runtime.pbit_queue!r}\n",
    ]
    with open("config_runtime.py", "w", encoding="utf-8") as f:
        f.writelines(lines)

from watchdog.events import FileSystemEventHandler
import os, config_runtime

class PbitHandler(FileSystemEventHandler):
    def on_created(self, event):
        if event.is_directory or not event.src_path.lower().endswith(".pbit"):
            return

        # only keep the filename, not the full path:
        nombre = os.path.basename(event.src_path)

        # avoid duplicates
        if nombre in config_runtime.pbit_queue:
            return

        config_runtime.pbit_queue.append(nombre)
        save_config_runtime()
        safe_print(f"📑 Cola actualizada: {config_runtime.pbit_queue}")



In [ ]:
# Celda 1: Detect & enqueue .pbit filenames only
import time
from watchdog.observers import Observer
import config, config_runtime

# init queue if needed
if not hasattr(config_runtime, "pbit_queue"):
    config_runtime.pbit_queue = []
    save_config_runtime()

observer = Observer()
observer.schedule(PbitHandler(), path=config.base_report_directory, recursive=False)
observer.start()
safe_print(f"👀 Observing {config.base_report_directory} (Ctrl+C to stop)")

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    observer.stop()
observer.join()


In [ ]:
# Celda 3: Script 1 + Two_A para cada archivo en cola, mostrando todo con safe_print()

import os
import config
import config_runtime

for nombre in config_runtime.pbit_queue:
    config_runtime.reporte = nombre
    save_config_runtime()

    full_path = os.path.join(config.base_report_directory, nombre)
    safe_print(f"\n🔄 Procesando: {full_path}")

    # Aquí mostramos TODA la salida, sin errores de decoding
    safe_print(ejecutar_script(config.script_one_path))
    safe_print(ejecutar_script(config.script_two_a_path))


In [ ]:
# Celda 5: Publicación final (Three)
safe_print("\n🚀 Ejecutando publicación final (Three)")
ejecutar_script(config.script_three_path)
